# Ride analysis of cycling fit files


## Import and chores


In [2]:
# --- Fix import paths so `import gpxfun.*` works from inside notebooks/ ---
import sys, os
from pathlib import Path

# Resolve the absolute path of the project root (one level up from 'notebooks/')
project_root = Path(os.getcwd()).resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✅ Project root added to sys.path:", project_root)


✅ Project root added to sys.path: C:\Users\betti\OneDrive\Dokumente\lukas_github\gpxfun


In [3]:
# Standard library
from pathlib import Path  # convenient, cross-platform path handling

# Data wrangling
import pandas as pd       # tabular data
import numpy as np        # fast numeric ops

# Geometry / CRS
import geopandas as gpd   # geospatial dataframes
from pyproj import Geod   # accurate geodesic distances using ellipsoid

# Local project modules
from gpxfun.io_fit import parse_fit                 # your FIT → DataFrame parser
from gpxfun.geo_utils import build_line_gdf, pick_utm_crs, to_projected  # helpers

# Project paths (relative to notebooks/)
RIDES_DIR = Path("../data/rides")   # folder with your .fit files
OUT_DIR   = Path("../outputs")      # where we’ll save QA tables and maps
OUT_DIR.mkdir(parents=True, exist_ok=True)  # create outputs/ if it doesn't exist

# Find a .fit to work with (first one for now)
fits = sorted(RIDES_DIR.rglob("*.fit"))   # recursively list .fit files
assert fits, f"No .fit files found under {RIDES_DIR.resolve()}"  # hard-stop if none
fit_path = fits[0]  # pick the first file deterministically
fit_path


WindowsPath('../data/rides/Blind_passenger_🐈.fit')

In [5]:
import requests, geopandas as gpd
from shapely.geometry import shape

def query_overpass(bbox: tuple[float,float,float,float], filter_expr: str) -> gpd.GeoDataFrame:
    """
    Query Overpass API directly and return a GeoDataFrame.
    bbox = (south, west, north, east)
    filter_expr example: way["bridge"="yes"]
    """
    overpass_url = "https://overpass-api.de/api/interpreter"
    south, west, north, east = bbox

    query = f"""
    [out:json][timeout:25];
    (
      {filter_expr}({south},{west},{north},{east});
    );
    out geom;
    """

    print("⏳ Sending query to Overpass...")
    r = requests.post(overpass_url, data={"data": query}, timeout=40)
    r.raise_for_status()

    data = r.json()
    elements = data.get("elements", [])
    print(f"✅ Received {len(elements)} elements")

    if not elements:
        return gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs="EPSG:4326")

    features = []
    for el in elements:
        if "geometry" not in el:
            continue
        coords = [(p["lon"], p["lat"]) for p in el["geometry"]]
        try:
            geom = shape({"type": "LineString", "coordinates": coords})
            features.append(geom)
        except Exception:
            continue
    return gpd.GeoDataFrame(geometry=features, crs="EPSG:4326")


## File ingestion and basic stats

In [6]:
# Parse the selected .fit into a raw DataFrame
raw = parse_fit(str(fit_path))  # returns columns like lat, lon, timestamp, altitude, hr, cadence, speed, ...

# Basic inspection to understand fields and size
display(raw.head(3))         # show first rows to confirm expected columns
print("rows:", len(raw))     # total record count
print("columns:", list(raw.columns))  # what sensors we got


,altitude,distance,enhanced_altitude,enhanced_speed,lat,lon,power,speed,temperature,timestamp
0,167.6,319.2,167.6,8.28,23.363388,121.319783,0,8.28,24,2025-10-11 05:20:18
1,167.2,327.7,167.2,8.40,23.363436,121.319847,0,8.40,23,2025-10-11 05:20:19
2,166.8,335.7,166.8,8.34,23.363483,121.319908,0,8.34,23,2025-10-11 05:20:20


rows: 17660
columns: ['altitude', 'distance', 'enhanced_altitude', 'enhanced_speed', 'lat', 'lon', 'power', 'speed', 'temperature', 'timestamp']


In [7]:
# --- Select the best available core fields ---
alt_col = "enhanced_altitude" if "enhanced_altitude" in raw.columns else "altitude"  # elevation in meters
spd_col = "enhanced_speed" if "enhanced_speed" in raw.columns else "speed"           # speed in m/s

# --- Build column list, making sure temperature & HR are included if present ---
keep_cols = ["timestamp", "lat", "lon"]
for col in [alt_col, spd_col, "temperature", "heart_rate", "power"]:
    if col in raw.columns:
        keep_cols.append(col)

# --- Work on a clean copy ---
pts = raw[keep_cols].copy()

# --- Rename to our standard schema (for consistency across devices) ---
rename_map = {
    alt_col: "ele",          # elevation
    spd_col: "speed_m_s",    # meters per second
}
pts = pts.rename(columns=rename_map)

# --- Time handling ---
pts["time"] = pd.to_datetime(pts["timestamp"], errors="coerce")  # ensure datetime
pts = pts.drop(columns=["timestamp"], errors="ignore")           # drop redundant field
pts = pts.sort_values("time").reset_index(drop=True)             # chronological order

# --- Drop rows missing mandatory spatial fields ---
pts = pts.dropna(subset=["lat", "lon", "time"])

# --- Add identifiers ---
ride_id = Path(fit_path).stem  # use filename as stable ride id
pts["ride_id"] = ride_id
pts["seq"] = np.arange(len(pts))  # sequential index for joining and plotting

# --- Display structure ---
display(pts.head(3))
print("✅ Cleaned rows:", len(pts))
print("columns:", list(pts.columns))


,lat,lon,ele,speed_m_s,temperature,power,time,ride_id,seq
0,23.363388,121.319783,167.6,8.28,24,0,2025-10-11 05:20:18,Blind_passenger_🐈,0
1,23.363436,121.319847,167.2,8.40,23,0,2025-10-11 05:20:19,Blind_passenger_🐈,1
2,23.363483,121.319908,166.8,8.34,23,0,2025-10-11 05:20:20,Blind_passenger_🐈,2


✅ Cleaned rows: 17660
columns: ['lat', 'lon', 'ele', 'speed_m_s', 'temperature', 'power', 'time', 'ride_id', 'seq']


In [8]:
# Each ride (i.e., each FIT file) gets a stable unique identifier
from pathlib import Path

# Derive a readable ride_id from the filename (e.g. "2025-08-05_morningride")
ride_id = Path(fit_path).stem

# Attach to all rows — allows grouping, joins, exports later
pts["ride_id"] = ride_id

# Add a sequential index to preserve order and enable joins
pts["seq"] = np.arange(len(pts))  # 0, 1, 2, ...

# Optionally preview — useful for debugging when we process multiple rides later
print(f"✅ ride_id = {ride_id}  |  total points = {len(pts)}")
display(pts.head(3)[["ride_id", "seq", "lat", "lon", "time"]])


✅ ride_id = Blind_passenger_🐈  |  total points = 17660


,ride_id,seq,lat,lon,time
0,Blind_passenger_🐈,0,23.363388,121.319783,2025-10-11 05:20:18
1,Blind_passenger_🐈,1,23.363436,121.319847,2025-10-11 05:20:19
2,Blind_passenger_🐈,2,23.363483,121.319908,2025-10-11 05:20:20


In [9]:
# --- Geodesic distance & time delta computation ---
from pyproj import Geod
import numpy as np

# Initialize geodesic calculator using WGS84 ellipsoid (accurate for Earth)
geod = Geod(ellps="WGS84")

# Extract coordinates and timestamps as numpy arrays for vectorized math
lat = pts["lat"].values
lon = pts["lon"].values
t  = pts["time"].values.astype("datetime64[ns]")  # nanosecond precision timestamps

# Compute distance between consecutive GPS points (in meters)
# geod.inv returns (azimuth_fwd, azimuth_back, distance_m)
_, _, dist_m = geod.inv(lon[:-1], lat[:-1], lon[1:], lat[1:])

# prepend 0 for the first row (no previous point)
pts["dist_m"] = np.r_[0.0, dist_m]

# Compute time differences between points (seconds)
# np.diff gives deltas; prepend NaN for first row
pts["dt_s"] = np.diff(t.astype("int64") / 1e9, prepend=np.nan)  # 1e9 converts ns → s

# Derive geometric speed (m/s) as distance / time
speed_geom = pts["dist_m"] / pts["dt_s"]

# Clean up impossible values (dt=0 → inf, NaN)
speed_geom = np.where(np.isfinite(speed_geom), speed_geom, np.nan)

# Combine: prefer device-recorded speed if valid, otherwise fallback to geometric one
pts["speed_m_s"] = np.where(
    (pts["speed_m_s"].notna()) & (pts["speed_m_s"] > 0),
    pts["speed_m_s"],  # keep device-recorded
    speed_geom,        # fallback
)

# Replace remaining NaNs/infinities with 0 (e.g., pauses)
pts["speed_m_s"] = np.nan_to_num(pts["speed_m_s"], nan=0.0, posinf=0.0, neginf=0.0)

# --- Quick sanity check ---
display(pts[["seq", "time", "dist_m", "dt_s", "speed_m_s"]].head(10))
print(f"✅ Distance/time metrics added — total rows: {len(pts)}")


,seq,time,dist_m,dt_s,speed_m_s
0,0,2025-10-11 05:20:18,0.000000,NaN,8.28
1,1,2025-10-11 05:20:19,8.430018,1.0,8.40
2,2,2025-10-11 05:20:20,8.126881,1.0,8.34
3,3,2025-10-11 05:20:21,7.362914,1.0,8.26
4,4,2025-10-11 05:20:22,7.826308,1.0,8.06
5,5,2025-10-11 05:20:23,7.588110,1.0,7.92
6,6,2025-10-11 05:20:24,8.120929,1.0,7.84
7,7,2025-10-11 05:20:25,7.774852,1.0,7.78
8,8,2025-10-11 05:20:26,7.781720,1.0,7.80
9,9,2025-10-11 05:20:27,8.047837,1.0,7.86


✅ Distance/time metrics added — total rows: 17660


In [10]:
# --- Bearing (direction of travel) computation ---
from pyproj import Geod
geod = Geod(ellps="WGS84")  # reuse the same ellipsoid model

# Compute forward azimuth (bearing) for each segment
# geod.inv returns (az12, az21, dist_m); we only need az12 here
az12, _, _ = geod.inv(lon[:-1], lat[:-1], lon[1:], lat[1:])

# Normalize bearings to 0–360° and align them to the DataFrame
bearing_deg = (az12 + 360) % 360
pts["heading_deg"] = np.r_[bearing_deg[0], bearing_deg]  # duplicate first bearing for length match

# --- Quick peek ---
display(pts[["seq", "heading_deg", "speed_m_s", "dist_m"]].head(10))
print("✅ Heading added — 0°=N, 90°=E, 180°=S, 270°=W")


,seq,heading_deg,speed_m_s,dist_m
0,0,50.959505,8.28,0.000000
1,1,50.959505,8.40,8.430018
2,2,50.148555,8.34,8.126881
3,3,48.555034,8.26,7.362914
4,4,50.435775,8.06,7.826308
5,5,47.808000,7.92,7.588110
6,6,50.198796,7.84,8.120929
7,7,53.260656,7.78,7.774852
8,8,53.298392,7.80,7.781720
9,9,58.421081,7.86,8.047837


✅ Heading added — 0°=N, 90°=E, 180°=S, 270°=W


In [11]:
# --- Ride summary (raw elevation) -------------------------------------------
import numpy as np
import pandas as pd

# --- Elevation gain ---------------------------------------------------------
if "ele" in pts.columns:
    delta_ele = pts["ele"].diff().fillna(0.0)
    elev_gain_m = float(delta_ele.clip(lower=0).sum())
else:
    elev_gain_m = np.nan

# --- Distance & timing ------------------------------------------------------
total_km = pts["dist_m"].sum() / 1000.0
moving_mask = pts["speed_m_s"] >= 1.0
moving_time_s = float(pts.loc[moving_mask, "dt_s"].fillna(0).clip(lower=0).sum())

start_time = pts["time"].min()
end_time   = pts["time"].max()
elapsed_s  = (end_time - start_time).total_seconds()

avg_moving_kmh = (pts.loc[moving_mask, "dist_m"].sum() / 1000.0) / (moving_time_s / 3600.0)

# --- Environmental / physiological -----------------------------------------
avg_hr = float(pts["heart_rate"].mean()) if "heart_rate" in pts.columns else np.nan
avg_temp_c = float(pts["temperature"].mean()) if "temperature" in pts.columns else np.nan

# --- Assemble summary table -------------------------------------------------
summary = pd.DataFrame([{
    "ride_id": pts["ride_id"].iloc[0],
    "start_time": start_time,
    "end_time": end_time,
    "elapsed_s": round(elapsed_s, 1),
    "moving_time_s": round(moving_time_s, 1),
    "total_km": round(total_km, 3),
    "elev_gain_m": round(elev_gain_m, 1),
    "avg_moving_kmh": round(avg_moving_kmh, 2),
    "avg_hr": round(avg_hr, 1) if not np.isnan(avg_hr) else None,
    "avg_temp_c": round(avg_temp_c, 1) if not np.isnan(avg_temp_c) else None,
}])

display(summary)
print("✅ ride_summary computed (raw elevation only)")


,ride_id,start_time,end_time,elapsed_s,moving_time_s,total_km,elev_gain_m,avg_moving_kmh,avg_hr,avg_temp_c
0,Blind_passenger_🐈,2025-10-11 05:20:18,2025-10-11 14:42:27,33729.0,17422.0,115.261,1237.0,23.65,None,25.0


✅ ride_summary computed (raw elevation only)


## 2. Ride display


In [12]:
import folium
from folium import plugins

# --- Compute ride center for initial map view ---
lat_center = pts["lat"].mean()
lon_center = pts["lon"].mean()

# --- Create a Folium map centered on your ride ---
m = folium.Map(
    location=[lat_center, lon_center],
    zoom_start=11,
    tiles="OpenStreetMap"
)



# --- Add the full ride trace as a polyline ---
coords = list(zip(pts["lat"], pts["lon"]))
folium.PolyLine(
    coords,
    color="red",
    weight=3,
    opacity=0.9,
    tooltip=f"Ride: {pts['ride_id'].iloc[0]}",
).add_to(m)

# --- Optional: add start & end markers ---
folium.Marker(
    location=[pts["lat"].iloc[0], pts["lon"].iloc[0]],
    popup="Start",
    icon=folium.Icon(color="green", icon="play"),
).add_to(m)

folium.Marker(
    location=[pts["lat"].iloc[-1], pts["lon"].iloc[-1]],
    popup="End",
    icon=folium.Icon(color="red", icon="stop"),
).add_to(m)

# --- Optional: add a minimap for orientation ---
plugins.MiniMap(toggle_display=True).add_to(m)

# --- Display the map interactively ---
m.save("ride_map.html")
import webbrowser
webbrowser.open("ride_map.html")



True

## Advanced OSM Metrics


In [13]:
from gpxfun.geo_utils import build_line_gdf, pick_utm_crs, to_projected

# Build LineString from your ride points (EPSG:4326)
gdf_line = build_line_gdf(pts)
print(gdf_line)

# Choose the best local UTM projection (for accurate meter-based ops)
utm_crs = pick_utm_crs(gdf_line)
print("Chosen UTM CRS:", utm_crs)

# Project to meters CRS (required for buffering and OSM overlays)
gdf_line_p = to_projected(gdf_line, utm_crs)
gdf_line_p


   ride_id                                           geometry
0        0  LINESTRING (121.31978 23.36339, 121.31985 23.3...
Chosen UTM CRS: EPSG:32651


,ride_id,geometry
0,0,"LINESTRING (328256.182 2584746.46, 328262.791 ..."


## Query overpass for bridges and tunnels

In [25]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests, time, folium, os
from shapely.geometry import LineString, box
from folium import plugins

# =============================================================================
# PARAMETERS
# =============================================================================
CHUNK_LEN_KM     = 40       # segment length per Overpass query (~10 km)
SEARCH_RADIUS_M  = 25       # buffer width around route (corridor width)
OVERLAP_M        = 200      # overlap between chunks (prevents gaps)
END_PAD_M        = 100      # pad beyond last ride point

# Overpass configuration
OVERPASS_MIRRORS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
]
OVERPASS_TIMEOUT = 60       # per request (s)
INITIAL_BACKOFF  = 5        # starting delay (s)
MAX_BACKOFF      = 300      # max delay (s)
MAX_WAIT_MIN     = 45       # abort after this many minutes

# Map rendering
MAP_ZOOM_START   = 11
MAP_TILE_STYLE   = "OpenStreetMap"
MAP_SAVE_PATH    = "ride_bridges_tunnels_final.html"

# Cache
CACHE_DIR        = "outputs/overpass_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================
def query_overpass_until_success(bbox, query_str, cache_name, label=""):
    """Query Overpass API with mirror rotation, caching, and retry-until-success logic."""
    cache_path = os.path.join(CACHE_DIR, cache_name)

    # --- Skip if cached ---
    if os.path.exists(cache_path):
        try:
            gdf_cached = gpd.read_file(cache_path)
            if not gdf_cached.empty:
                print(f"🟢 Using cached {label} result for {cache_name} ({len(gdf_cached)} features)")
                return gdf_cached
        except Exception:
            print(f"⚠️ Cache file {cache_name} unreadable, refetching...")

    # --- Prepare Overpass query ---
    south, west, north, east = bbox
    q = f"""
    [out:json][timeout:{OVERPASS_TIMEOUT}];
    (
      {query_str}({south},{west},{north},{east});
    );
    out geom;
    """

    attempt = 1
    delay = INITIAL_BACKOFF
    total_wait = 0
    mirror_index = 0

    while True:
        url = OVERPASS_MIRRORS[mirror_index % len(OVERPASS_MIRRORS)]
        print(f"⏳ [{label}] Attempt {attempt} via {url} ...")

        try:
            r = requests.post(url, data={"data": q}, timeout=OVERPASS_TIMEOUT)
            r.raise_for_status()
            data = r.json()
            elements = data.get("elements", [])

            # --- Legitimate empty result (no retry) ---
            if not elements:
                print(f"ℹ️ No {label} features found for {cache_name} (valid empty response).")
                empty_gdf = gpd.GeoDataFrame(columns=["geometry"], crs=4326)
                empty_gdf.to_file(cache_path, driver="GeoJSON")
                return empty_gdf

            # --- Parse features ---
            features = []
            for el in elements:
                if "geometry" in el and el["type"] == "way":
                    coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
                    features.append({
                        "type": "Feature",
                        "geometry": LineString(coords).__geo_interface__,
                        "properties": {"osmid": el.get("id")}
                    })

            gdf = gpd.GeoDataFrame.from_features(features, crs="EPSG:4326")
            gdf.to_file(cache_path, driver="GeoJSON")
            print(f"✅ [{label}] Success ({len(gdf)} features) — cached as {cache_name}")
            return gdf

        except requests.exceptions.HTTPError as e:
            print(f"⚠️ [{label}] HTTP {e.response.status_code}: {e.response.reason}")
        except requests.exceptions.RequestException as e:
            print(f"⚠️ [{label}] Network error: {e}")
        except Exception as e:
            print(f"⚠️ [{label}] Unexpected error: {type(e).__name__}: {e}")

        # --- Retry logic ---
        time.sleep(delay)
        total_wait += delay / 60
        delay = min(delay * 1.5, MAX_BACKOFF)
        attempt += 1
        mirror_index += 1  # rotate mirror

        if total_wait > MAX_WAIT_MIN:
            print(f"❌ [{label}] Aborted after {MAX_WAIT_MIN} minutes total wait for {cache_name}")
            return gpd.GeoDataFrame(columns=["geometry"], crs=4326)


def dedup_geoms(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Remove duplicate OSM features across overlapping chunks."""
    if gdf.empty:
        return gdf
    before = len(gdf)
    if "osmid" in gdf.columns:
        gdf = gdf.drop_duplicates(subset="osmid")
    else:
        gdf["wkt"] = gdf.geometry.to_wkt()
        gdf = gdf.drop_duplicates(subset="wkt").drop(columns="wkt")
    after = len(gdf)
    if before > after:
        print(f"ℹ️ Removed {before - after} duplicate geometries")
    return gdf.reset_index(drop=True)

# =============================================================================
# STEP 1: Prepare ride geometry
# =============================================================================
utm_crs = pick_utm_crs(gdf_line)
line_utm = gdf_line.to_crs(utm_crs)
total_len_m = line_utm.length.iloc[0]
chunk_len_m = CHUNK_LEN_KM * 1000
n_chunks = int(np.ceil(total_len_m / chunk_len_m))
print(f"Total ride {total_len_m/1000:.1f} km → {n_chunks} chunks")

# =============================================================================
# STEP 2: Prepare all points with distance along ride
# =============================================================================
pts_gdf = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts.lon, pts.lat), crs=4326).to_crs(utm_crs)
line_geom = line_utm.geometry.iloc[0]
pts_gdf["dist_m"] = pts_gdf.geometry.apply(line_geom.project)

bridges_all, tunnels_all = [], []
bbox_rects = []

# =============================================================================
# STEP 3: Iterate through chunks and query Overpass (with caching)
# =============================================================================
for i in range(n_chunks):
    start_d = max(0, i * chunk_len_m - OVERLAP_M)
    end_d   = min((i + 1) * chunk_len_m + OVERLAP_M, total_len_m + END_PAD_M)

    seg_pts = pts_gdf[(pts_gdf["dist_m"] >= start_d) & (pts_gdf["dist_m"] <= end_d)]
    if seg_pts.empty:
        print(f"⚠️ Chunk {i+1} empty, skipping")
        continue

    # true bbox of actual GPS points in this segment
    minx, miny, maxx, maxy = seg_pts.total_bounds
    pad = SEARCH_RADIUS_M * 2
    minx -= pad; miny -= pad; maxx += pad; maxy += pad

    bbox_wgs = gpd.GeoSeries([box(minx, miny, maxx, maxy)], crs=utm_crs).to_crs(4326).total_bounds
    west, south, east, north = bbox_wgs
    bbox = (south, west, north, east)
    bbox_rects.append((west, south, east, north))

    print(f"\n--- Chunk {i+1}/{n_chunks} ---")
    print(f"lon[{west:.3f},{east:.3f}] lat[{south:.3f},{north:.3f}]")

    g_b = query_overpass_until_success(bbox, 'way["bridge"="yes"]["highway"]',
                                       f"bridges_chunk_{i+1:02}.geojson", label="bridges")
    g_t = query_overpass_until_success(bbox, 'way["tunnel"="yes"]["highway"]',
                                       f"tunnels_chunk_{i+1:02}.geojson", label="tunnels")

    if not g_b.empty: bridges_all.append(g_b)
    if not g_t.empty: tunnels_all.append(g_t)

# =============================================================================
# STEP 4: Merge & deduplicate results
# =============================================================================
g_bridges = pd.concat(bridges_all, ignore_index=True) if bridges_all else gpd.GeoDataFrame(columns=["geometry"], crs=4326)
g_tunnels = pd.concat(tunnels_all, ignore_index=True) if tunnels_all else gpd.GeoDataFrame(columns=["geometry"], crs=4326)

g_bridges = dedup_geoms(g_bridges)
g_tunnels = dedup_geoms(g_tunnels)

# Filter to ride corridor
buf_corridor = line_utm.buffer(SEARCH_RADIUS_M).to_crs(4326).geometry.iloc[0]
if not g_bridges.empty:
    g_bridges = g_bridges[g_bridges.intersects(buf_corridor)]
if not g_tunnels.empty:
    g_tunnels = g_tunnels[g_tunnels.intersects(buf_corridor)]

print(f"\n✅ Bridges in {SEARCH_RADIUS_M} m corridor: {len(g_bridges)} | tunnels: {len(g_tunnels)}")

# =============================================================================
# STEP 5: Map visualization
# =============================================================================
m = folium.Map(location=[pts["lat"].mean(), pts["lon"].mean()],
               zoom_start=MAP_ZOOM_START,
               tiles=MAP_TILE_STYLE)

# Ride line
folium.PolyLine(list(zip(pts["lat"], pts["lon"])),
                color="red", weight=3, opacity=0.9,
                tooltip=f"Ride: {pts['ride_id'].iloc[0]}").add_to(m)

# Corridor
folium.GeoJson(buf_corridor.__geo_interface__,
               style_function=lambda x: {"color": "orange", "weight": 1, "fillOpacity": 0.15}).add_to(m)

# Chunk rectangles (light gray shading)
for idx, (west, south, east, north) in enumerate(bbox_rects, start=1):
    folium.Rectangle(
        bounds=[[south, west], [north, east]],
        color="#555555",
        fill=True,
        fill_color="#999999",
        fill_opacity=0.35,
        weight=1,
        dash_array="4",
        tooltip=f"Chunk {idx}"
    ).add_to(m)

# Bridges (blue)
for g in g_bridges.geometry:
    folium.GeoJson(g.__geo_interface__,
                   style_function=lambda x: {"color": "blue", "weight": 3}).add_to(m)

# Tunnels (green)
for g in g_tunnels.geometry:
    folium.GeoJson(g.__geo_interface__,
                   style_function=lambda x: {"color": "green", "weight": 3}).add_to(m)

# MiniMap
plugins.MiniMap(toggle_display=True).add_to(m)

# Save & open
m.save(MAP_SAVE_PATH)
import webbrowser; webbrowser.open(MAP_SAVE_PATH)


Total ride 115.2 km → 3 chunks

--- Chunk 1/3 ---
lon[121.316,121.443] lat[23.362,23.629]
⏳ [bridges] Attempt 1 via https://overpass-api.de/api/interpreter ...
⚠️ [bridges] HTTP 504: Gateway Timeout
⏳ [bridges] Attempt 2 via https://overpass.kumi.systems/api/interpreter ...
✅ [bridges] Success (139 features) — cached as bridges_chunk_01.geojson
⏳ [tunnels] Attempt 1 via https://overpass-api.de/api/interpreter ...
✅ [tunnels] Success (15 features) — cached as tunnels_chunk_01.geojson

--- Chunk 2/3 ---
lon[121.434,121.562] lat[23.599,23.747]
⏳ [bridges] Attempt 1 via https://overpass-api.de/api/interpreter ...
✅ [bridges] Success (122 features) — cached as bridges_chunk_02.geojson
⏳ [tunnels] Attempt 1 via https://overpass-api.de/api/interpreter ...
✅ [tunnels] Success (3 features) — cached as tunnels_chunk_02.geojson

--- Chunk 3/3 ---
lon[121.552,121.612] lat[23.744,23.986]
⏳ [bridges] Attempt 1 via https://overpass-api.de/api/interpreter ...
✅ [bridges] Success (132 features) — cache

True